In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
from cpt.solver.train_critic import make_env, Solver, DATA_PATH, MODEL_PATH, BCDataset
train_dataset = BCDataset(os.path.join(DATA_PATH, f"move_val.h5"), return_value=True)

env = make_env('rope', **{"sim_cfg.max_steps": 100})

solver = Solver.parse(
    env, low_steps=40,
    parse_prefix='solver',
    head_cfg=dict(std_scale=0.001),
    actor_lr=0.001,
    n_sample_poses=1000,
    nn_critic_ratio=1000.,
    backbone=dict(TYPE="PointnetBackbone")
)

actor_before = torch.load(f'{MODEL_PATH}/move_pretrain/best_actor')
solver.actor.load_state_dict(actor_before.state_dict())

critic_before = torch.load(f'{MODEL_PATH}/online_critic/critic')
solver.critic.load_state_dict(critic_before.state_dict())

E: 5000.0
gravity: (0.0, -1.0, 0.0)
grid_size: (1.0, 1.0, 1.0)
ground_friction: 5.0
max_steps: 100
n_particles: 20000
nu: 0.2
yield_stress: 100.0
LOADING....
Loaded with 0.00030732154846191406 secs.
GPU 0/1 memory: free=10.341, total=10.917
init renderer


<All keys matched successfully>

In [3]:
import numpy as np
from cpt.solver.cut import sample_move
from diffrl.utils import animate
#for idx, data in zip(range(10), train_dataset):
#    print(data['0']['pointcloud'])
images = []
ious = []
for i in range(10):
    state, goal = sample_move(env)
    ious.append(solver.sample_trajectory(state, goal, images=images, num_stages=1)['loss'])

print(np.mean(ious))
animate(images)

96.82750253677368
animating animation.mp4
Moviepy - Building video animation.mp4.
Moviepy - Writing video animation.mp4



Moviepy - Done !
Moviepy - video ready animation.mp4


In [6]:
images = []
ious = []
for i in range(10):
    state, goal = sample_move(env)
    ious.append(solver.sample_trajectory(state, goal, images=images, num_stages=1, n_sample_poses=1)['loss'])

print(np.mean(ious))
animate(images, 'no_critic.mp4')

0


IndexError: list index out of range